In [235]:
import os
import numpy as np
import pandas as pd

from sklearn import model_selection

import tensorflow as tf
import tflearn

from sklearn.metrics import r2_score

# Reading the Data

In [236]:
datapath = 'Data/'
test_csv = 'test.csv'
train_csv = 'train.csv'
sample_csv = 'sample_submission.csv'

In [237]:
data = pd.read_csv(os.path.join(datapath,train_csv))

# Understanding the Data

In [238]:
print('Variables with letters are categorical. Variables with 0/1 are binary values.')
data.head(3)

Variables with letters are categorical. Variables with 0/1 are binary values.


,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0


In [239]:
tp_int = []
tp_float = []
tp_obj = []
tp_other = []

#Getting all variables names, less ID and looking for it types 
for i in data.columns.to_series()[1:]:
    if data[i].dtype == 'int64':
        tp_int.append(i)
    elif data[i].dtype == 'float64':
        tp_float.append(i)
    elif data[i].dtype == 'object':
        tp_obj.append(i)
    else:
        tp_other.append(i)
        
dic = {'tp_int': tp_int, 'tp_float': tp_float, 'tp_obj': tp_obj, 'tp_other': tp_other}

print('Categorical:', tp_obj)
print('Float:', tp_float)
print('Has other type?', tp_other)

Categorical: ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']
Float: ['y']
Has other type? []


In [240]:
drop = []
binarys = []
for i in tp_int:
    print('Variable: {}, Min: {}, Max: {} , Unique: {}'.format(i, data[i].min(), data[i].max() ,data[i].unique()))
    if data[i].max() == 0:
        drop.append(i)
    elif data[i].max() == 1 and data[i].min() == 0 and len(data[i].unique()) == 2:
        binarys.append(i)
    else:
        pass 

Variable: X10, Min: 0, Max: 1 , Unique: [0 1]
Variable: X11, Min: 0, Max: 0 , Unique: [0]
Variable: X12, Min: 0, Max: 1 , Unique: [0 1]
Variable: X13, Min: 0, Max: 1 , Unique: [1 0]
Variable: X14, Min: 0, Max: 1 , Unique: [0 1]
Variable: X15, Min: 0, Max: 1 , Unique: [0 1]
Variable: X16, Min: 0, Max: 1 , Unique: [0 1]
Variable: X17, Min: 0, Max: 1 , Unique: [0 1]
Variable: X18, Min: 0, Max: 1 , Unique: [1 0]
Variable: X19, Min: 0, Max: 1 , Unique: [0 1]
Variable: X20, Min: 0, Max: 1 , Unique: [0 1]
Variable: X21, Min: 0, Max: 1 , Unique: [1 0]
Variable: X22, Min: 0, Max: 1 , Unique: [0 1]
Variable: X23, Min: 0, Max: 1 , Unique: [0 1]
Variable: X24, Min: 0, Max: 1 , Unique: [0 1]
Variable: X26, Min: 0, Max: 1 , Unique: [0 1]
Variable: X27, Min: 0, Max: 1 , Unique: [0 1]
Variable: X28, Min: 0, Max: 1 , Unique: [0 1]
Variable: X29, Min: 0, Max: 1 , Unique: [0 1]
Variable: X30, Min: 0, Max: 1 , Unique: [0 1]
Variable: X31, Min: 0, Max: 1 , Unique: [1 0]
Variable: X32, Min: 0, Max: 1 , Uniq

Variable: X219, Min: 0, Max: 1 , Unique: [0 1]
Variable: X220, Min: 0, Max: 1 , Unique: [1 0]
Variable: X221, Min: 0, Max: 1 , Unique: [0 1]
Variable: X222, Min: 0, Max: 1 , Unique: [0 1]
Variable: X223, Min: 0, Max: 1 , Unique: [0 1]
Variable: X224, Min: 0, Max: 1 , Unique: [0 1]
Variable: X225, Min: 0, Max: 1 , Unique: [0 1]
Variable: X226, Min: 0, Max: 1 , Unique: [0 1]
Variable: X227, Min: 0, Max: 1 , Unique: [0 1]
Variable: X228, Min: 0, Max: 1 , Unique: [0 1]
Variable: X229, Min: 0, Max: 1 , Unique: [0 1]
Variable: X230, Min: 0, Max: 1 , Unique: [0 1]
Variable: X231, Min: 0, Max: 1 , Unique: [0 1]
Variable: X232, Min: 0, Max: 1 , Unique: [0 1]
Variable: X233, Min: 0, Max: 0 , Unique: [0]
Variable: X234, Min: 0, Max: 1 , Unique: [1 0]
Variable: X235, Min: 0, Max: 0 , Unique: [0]
Variable: X236, Min: 0, Max: 1 , Unique: [0 1]
Variable: X237, Min: 0, Max: 1 , Unique: [1 0]
Variable: X238, Min: 0, Max: 1 , Unique: [0 1]
Variable: X239, Min: 0, Max: 1 , Unique: [0 1]
Variable: X240, M

In [241]:
print('Variables only with 0:', drop)

if len(drop) + len(binarys) == len(tp_int):
    print('All other integer variables are binary')

Variables only with 0: ['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293', 'X297', 'X330', 'X347']
All other integer variables are binary


In [242]:
#Dropping variables only with zero
data = data.drop(drop, axis = 1)

# One hot enconding

In [243]:
#Best way to work with categorical variables? I don't think is the best in all case. I will study about that

for each in tp_obj:
    dummies = pd.get_dummies(data[each], prefix=each, drop_first=False)
    data = pd.concat([data, dummies], axis=1)

data = data.drop(tp_obj, axis=1)
data.head()

,ID,y,X10,X12,X13,X14,X15,X16,X17,X18,...,X8_p,X8_q,X8_r,X8_s,X8_t,X8_u,X8_v,X8_w,X8_x,X8_y
0,0,130.81,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,6,88.53,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,7,76.26,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,9,80.62,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Test Data

In [244]:
data_test = pd.read_csv(os.path.join(datapath,test_csv))

In [245]:
#Dropping variables only with zero
data_test = data_test.drop(drop, axis = 1)

In [246]:
for each in tp_obj:
    dummies = pd.get_dummies(data_test[each], prefix=each, drop_first=False)
    data_test = pd.concat([data_test, dummies], axis=1)

data_test = data_test.drop(tp_obj, axis=1)
data_test.head()

,ID,X10,X12,X13,X14,X15,X16,X17,X18,X19,...,X8_p,X8_q,X8_r,X8_s,X8_t,X8_u,X8_v,X8_w,X8_x,X8_y
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,2,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,3,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Keeping only columns who are in both data sets

In [247]:
keep = np.intersect1d(data.columns.to_series(), data_test.columns.to_series())

In [248]:
data = data[np.append(keep,'y')]

In [249]:
data_test = data_test[keep]

# PCA and ICA

In [250]:
## Got this from https://www.kaggle.com/frednavruzov/baselines-to-start-with-lb-0-56

from sklearn.decomposition import PCA, FastICA
n_comp = 10

# PCA
pca = PCA(n_components=n_comp, random_state=42)
pca2_results_train = pca.fit_transform(data.drop(["y"], axis=1))
pca2_results_test = pca.transform(data_test)

# ICA
ica = FastICA(n_components=n_comp, random_state=42)
ica2_results_train = ica.fit_transform(data.drop(["y"], axis=1))
ica2_results_test = ica.transform(data_test)

# Append decomposition components to datasets
for i in range(1, n_comp+1):
    data['pca_' + str(i)] = pca2_results_train[:,i-1]
    data_test['pca_' + str(i)] = pca2_results_test[:, i-1]
    
    data['ica_' + str(i)] = ica2_results_train[:,i-1]
    data_test['ica_' + str(i)] = ica2_results_test[:, i-1]
    


In [ ]:
#pca_drop = list(data)[1:-101]
#print(pca_drop)
#print(type(pca_drop))

In [ ]:
#The guy in the link is only adding new colunms. I think this doesnt make sense (and barely changed the result). 

#I also created more PCA/ICA features than just 10

#data = data.drop(labels = pca_drop, axis = 1)
#data_test = data_test.drop(labels = pca_drop, axis = 1)

#print(data.head())
#print(data_test.head())

#This got worse results


# Splitting data into training/testing

In [251]:
train_features, test_features, train_y, test_y = model_selection.train_test_split(
    data, data['y'], test_size = 0.3, random_state = 0)

In [252]:
train_features = train_features.drop(['ID', 'y'], axis = 1).values
test_features = test_features.drop(['ID', 'y'], axis = 1).values

train_y = train_y[:, np.newaxis]

In [253]:
test_features.shape

(1263, 561)

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

randomforest = RandomForestRegressor(n_estimators=200, max_features='auto', bootstrap=False, 
                                   oob_score=False, n_jobs=-1, random_state=0).fit(train_features, train_y)

randomforest_score = randomforest.score(test_features, test_y)
print('RF Score:', randomforest_score)

predict = randomforest.predict(test_features)

slope, intercept, r_value, p_value, std_err = stats.linregress(predict, test_y)
print('R Square:', r_value**2)

In [ ]:
predict_test = randomforest.predict(data_test.iloc[:, 1:])

In [ ]:
submission = pd.DataFrame({'ID': data_test['ID'], 'y': predict_test})
submission.to_csv(os.path.join(datapath,'submissionrf.csv'), index = False)

# Rede Neural

In [ ]:
import tensorflow as tf
import tflearn

# Define the neural network
def build_model():
    # This resets all parameters and variables, leave this here
    tf.reset_default_graph()
    
    # Inputs
    net = tflearn.input_data([None, train_features.shape[1]])

    # Hidden layer(s)
    net = tflearn.fully_connected(net, 800, activation='ReLU')
    net = tflearn.dropout(net, 0.80)  
    
    net = tflearn.fully_connected(net, 1200, activation='ReLU')
    net = tflearn.dropout(net, 0.80)
    
    net = tflearn.fully_connected(net, 800, activation='ReLU')
    net = tflearn.dropout(net, 0.80)

    net = tflearn.fully_connected(net, 400, activation='ReLU')
    net = tflearn.dropout(net, 0.80)    
    
    # Output layer and training model
    net = tflearn.fully_connected(net, 1, activation='linear')
    net = tflearn.regression(net, optimizer='sgd', learning_rate=0.1, loss="mean_square")
    
    model = tflearn.DNN(net)
    return model

model = build_model()

In [ ]:
import tensorflow as tf
import tflearn

# Define the neural network
def build_model():
    # This resets all parameters and variables, leave this here
    tf.reset_default_graph()
    
    # Inputs
    net = tflearn.input_data([None, train_features.shape[1]])

    # Hidden layer(s)
    net = tflearn.fully_connected(net, 512, activation='ReLU') 
    net = tflearn.fully_connected(net, 512, activation='ReLU')
    net = tflearn.dropout(net, 0.80)
    
    # Output layer and training model
    net = tflearn.fully_connected(net, 1, activation='linear')
    net = tflearn.regression(net, optimizer='sgd', learning_rate=0.1, loss="mean_square")
    
    model = tflearn.DNN(net)
    return model

model = build_model()

In [ ]:
model.fit(train_features, train_y, validation_set=0.2, show_metric=True, batch_size=512, n_epoch=10000)

In [ ]:
predict = model.predict(test_features)
predict = [predict[i][0] for i in range(0, len(predict))]

slope, intercept, r_value, p_value, std_err = stats.linregress(predict, test_y)
print('R Square:', r_value**2)

In [ ]:
predict_test = model.predict(data_test.iloc[:, 1:])
predict_test = [predict_test[i][0] for i in range(0, len(predict_test))]

In [ ]:
submission = pd.DataFrame({'ID': data_test['ID'], 'y': predict_test})
print(submission.head())
submission.to_csv(os.path.join(datapath,'submissiondp.csv'), index = False)

# xgboost

In [254]:
 ()# mmm, xgboost, loved by everyone ^-^
import xgboost as xgb

# prepare dict of params for xgboost to run with
xgb_params = {
    'n_trees': 500, 
    'eta': 0.005,
    'max_depth': 4,
    'subsample': 0.95,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': train_y.mean(), # base prediction = mean(target)
    'silent': 1
}

# form DMatrices for Xgboost training
dtrain = xgb.DMatrix(train_features, train_y)
dtest = xgb.DMatrix(data_test.iloc[:, 1:].values)

# xgboost, cross-validation
cv_result = xgb.cv(xgb_params, 
                   dtrain, 
                   num_boost_round=700, # increase to have better results (~700)
                   early_stopping_rounds=50,
                   verbose_eval=50, 
                   show_stdv=False
                  )

num_boost_rounds = len(cv_result)
print(num_boost_rounds)

# train model
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)

[0]	train-rmse:12.4241	test-rmse:12.424
[50]	train-rmse:10.8222	test-rmse:10.9072
[100]	train-rmse:9.69559	test-rmse:9.87839
[150]	train-rmse:8.91658	test-rmse:9.20208
[200]	train-rmse:8.38358	test-rmse:8.77176
[250]	train-rmse:8.01892	test-rmse:8.50778
[300]	train-rmse:7.76462	test-rmse:8.34992
[350]	train-rmse:7.5813	test-rmse:8.2602
[400]	train-rmse:7.44362	test-rmse:8.20867
[450]	train-rmse:7.32998	test-rmse:8.17726
[500]	train-rmse:7.2315	test-rmse:8.16091
[550]	train-rmse:7.15339	test-rmse:8.15137
[600]	train-rmse:7.08183	test-rmse:8.14459
[650]	train-rmse:7.01081	test-rmse:8.14621
624


In [255]:
print(r2_score(dtrain.get_label(), model.predict(dtrain)))

0.65725615563


In [256]:
# make predictions and save results
y_pred = model.predict(dtest)
submission = pd.DataFrame({'id': data_test['ID'], 'y': y_pred})
submission.to_csv(os.path.join(datapath,'submissionxg.csv'), index = False)